# Checkpoint Three: Cleaning Data

Now you are ready to clean your data. Before starting coding, provide the link to your dataset below.

My dataset:https://www.kaggle.com/datasets/abdallaahmed77/healthcare-risk-factors-dataset

Import the necessary libraries and create your dataframe(s)

In [ ]:
import pandas as pd

health_df = pd.read_csv("Healthcare_risk.csv")
# Want to see if the dataframe was properly imported and see what kind of data is stored in each column
print(health_df.info())
print(health_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                25500 non-null  float64
 1   Gender             25500 non-null  object 
 2   Medical Condition  25500 non-null  object 
 3   Glucose            25500 non-null  float64
 4   Blood Pressure     25500 non-null  float64
 5   BMI                30000 non-null  float64
 6   Oxygen Saturation  30000 non-null  float64
 7   LengthOfStay       30000 non-null  int64  
 8   Cholesterol        30000 non-null  float64
 9   Triglycerides      30000 non-null  float64
 10  HbA1c              30000 non-null  float64
 11  Smoking            30000 non-null  int64  
 12  Alcohol            30000 non-null  int64  
 13  Physical Activity  30000 non-null  float64
 14  Diet Score         30000 non-null  float64
 15  Family History     30000 non-null  int64  
 16  Stress Level       300

## Missing Data

Test your dataset for missing data and handle it as needed. Make notes in the form of code comments as to your thought process.

In [2]:
# Check for missing values in each column
missing_values = health_df.isnull().sum()
print(missing_values)

Age                  4500
Gender               4500
Medical Condition    4500
Glucose              4500
Blood Pressure       4500
BMI                     0
Oxygen Saturation       0
LengthOfStay            0
Cholesterol             0
Triglycerides           0
HbA1c                   0
Smoking                 0
Alcohol                 0
Physical Activity       0
Diet Score              0
Family History          0
Stress Level            0
Sleep Hours             0
random_notes            0
noise_col               0
dtype: int64


4500 missing rows  for:
Age, Gender, Medical Condition, Glucose, Blood Pressure.
All other columns have no missing data.
So for cleaning, I need to handle these five columns with missing data.

In [3]:
#Fill missing values
# Fill numeric columns with median
numeric_cols = ['Age', 'Glucose', 'Blood Pressure']
for col in numeric_cols:
    health_df[col].fillna(health_df[col].median(), inplace=True)

# Fill categorical columns with mode
categorical_cols = ['Gender', 'Medical Condition']
for col in categorical_cols:
    health_df[col].fillna(health_df[col].mode()[0], inplace=True)

In [4]:
# Verify no missing values remain
print(health_df.isnull().sum())

Age                  0
Gender               0
Medical Condition    0
Glucose              0
Blood Pressure       0
BMI                  0
Oxygen Saturation    0
LengthOfStay         0
Cholesterol          0
Triglycerides        0
HbA1c                0
Smoking              0
Alcohol              0
Physical Activity    0
Diet Score           0
Family History       0
Stress Level         0
Sleep Hours          0
random_notes         0
noise_col            0
dtype: int64


## Irregular Data

Detect outliers in your dataset and handle them as needed. Use code comments to make notes about your thought process.

In [6]:
# Check min and max values for numeric columns
numeric_cols = health_df.select_dtypes(include=['float64', 'int64']).columns

for col in numeric_cols:
    print(f"{col}: min = {health_df[col].min()}, max = {health_df[col].max()}")

Age: min = 10.0, max = 89.0
Glucose: min = 20.32, max = 318.51
Blood Pressure: min = 74.24, max = 226.38
BMI: min = 7.67, max = 56.85
Oxygen Saturation: min = 67.51, max = 110.07
LengthOfStay: min = 1, max = 19
Cholesterol: min = 95.73, max = 358.37
Triglycerides: min = -22.48, max = 421.51
HbA1c: min = 3.28, max = 12.36
Smoking: min = 0, max = 1
Alcohol: min = 0, max = 1
Physical Activity: min = -3.68, max = 12.41
Diet Score: min = -1.75, max = 12.06
Family History: min = 0, max = 1
Stress Level: min = -2.44, max = 15.45
Sleep Hours: min = 1.59, max = 10.35
noise_col: min = -412.1695960009927, max = 467.8949100376724


 Decide how to handle irregular values

1. Drop noise_col – it’s clearly just random numbers.
2. For numeric health metrics with impossible negative values (Triglycerides, Physical Activity, Diet Score, Stress Level):

 Replace negative values with median of the column.

3. Other high values like Glucose = 318 or BP = 226 can be kept if they might represent real extreme cases.

In [7]:
# Drop noise_col
health_df.drop(columns=['noise_col'], inplace=True)

# List of columns where negative values are impossible
cols_to_fix = ['Triglycerides', 'Physical Activity', 'Diet Score', 'Stress Level']

# Replace negative values with median
for col in cols_to_fix:
    median_value = health_df[col].median()
    health_df[col] = health_df[col].apply(lambda x: median_value if x < 0 else x)

In [8]:
# Verify
for col in cols_to_fix:
    print(f"{col}: min = {health_df[col].min()}, max = {health_df[col].max()}")

Triglycerides: min = 1.14, max = 421.51
Physical Activity: min = -0.0, max = 12.41
Diet Score: min = 0.0, max = 12.06
Stress Level: min = -0.0, max = 15.45


## Unnecessary Data

Look for the different types of unnecessary data in your dataset and address it as needed. Make sure to use code comments to illustrate your thought process.

In [12]:
 #Drop unnecessary columns
 #'random_notes' is free text, not useful for analysis
health_df.drop(columns=['random_notes'], inplace=True)

# Verify columns left
print(health_df.columns)

Index(['Age', 'Gender', 'Medical Condition', 'Glucose', 'Blood Pressure',
       'BMI', 'Oxygen Saturation', 'LengthOfStay', 'Cholesterol',
       'Triglycerides', 'HbA1c', 'Smoking', 'Alcohol', 'Physical Activity',
       'Diet Score', 'Family History', 'Stress Level', 'Sleep Hours'],
      dtype='object')


## Inconsistent Data

Check for inconsistent data and address any that arises. As always, use code comments to illustrate your thought process.

Categorical columns:

Misspellings, different capitalizations, or unexpected values.

Example: "Male" vs "male" vs "M" in Gender.

Numeric columns:

Sometimes values are in the wrong scale or units, e.g., Glucose in mg/dL vs mmol/L 

Can also be unrealistic but technically not negative ( handled that).

In [14]:
#Check categorical columns for inconsistencies
# Check unique values in categorical columns
categorical_cols = ['Gender', 'Medical Condition']

for col in categorical_cols:
    print(f"Unique values in {col}: {health_df[col].unique()}")

Unique values in Gender: ['Male' 'Female']
Unique values in Medical Condition: ['Diabetes' 'Healthy' 'Asthma' 'Obesity' 'Hypertension' 'Cancer'
 'Arthritis']


In [15]:
# Check frequency of each unique value
print(health_df['Medical Condition'].value_counts())

Medical Condition
Hypertension    11620
Diabetes         6417
Obesity          3857
Healthy          3039
Asthma           2037
Arthritis        1796
Cancer           1234
Name: count, dtype: int64


# Step 4: Inconsistent Data
# Checked unique values in categorical columns
# Gender has only 'Male' and 'Female'
# Medical Condition has consistent labels: 'Diabetes', 'Healthy', 'Asthma', 'Obesity', 'Hypertension', 'Cancer', 'Arthritis'
# No inconsistencies found. No further action required.


## Summarize Your Results

Make note of your answers to the following questions.

1. Did you find all four types of dirty data in your dataset?

 Yes, all four types were observed:
# - Missing Data: 'Age', 'Gender', 'Glucose', 'Blood Pressure', and 'Medical Condition' had missing values (~4500 rows).
# - Irregular Data / Outliers: Negative values in 'Triglycerides', 'Physical Activity', 'Diet Score', and 'Stress Level'. Extreme high values in 'Glucose' and 'Blood Pressure' were noted but plausible.
# - Unnecessary Data: 'noise_col' and 'random_notes' were removed as they were not useful for analysis.
# - Inconsistent Data: Checked 'Gender' and 'Medical Condition'; no inconsistencies were found. 

2. Did the process of cleaning your data give you new insights into your dataset?
Negative values in some numeric columns highlighted the need for careful data validation.
# - Extreme values like very high Glucose or Blood Pressure may represent rare but real cases; so I kept them.
# - Some categorical columns were already clean, showing good data quality in parts of the dataset.

3. Is there anything you would like to make note of when it comes to manipulating the data and making visualizations?

 Always check for negative or impossible values before plotting numeric data to avoid skewed visualizations.
# - Ensure categorical labels are consistent to create accurate plots.
# - Dropping unnecessary columns early simplifies data manipulation and makes visualization code cleaner.
# - Extreme values or outliers may require special handling in charts .
